In [ ]:
!pip install pytket

In [ ]:
import os
from os.path import join, isfile
import csv

from pytket import Circuit, OpType
from pytket.qasm import circuit_from_qasm, circuit_to_qasm
from pytket.passes import (
    FullPeepholeOptimise,
    RemoveRedundancies,
    PauliSimp,
    SequencePass,
    CliffordSimp,
    RebaseCustom
)

In [ ]:
def tket_optimize_nam(fn: str, output_dir: str):
    circ = circuit_from_qasm(fn)

    FullPeepholeOptimise().apply(circ)

    output_path = output_dir + fn.split("/")[-1]

    circuit_to_qasm(circ, output_path)

    return circ.n_gates, circ.n_2qb_gates(), circ.depth()


circuit_order = [
    "tof_3",
    "barenco_tof_3",
    "mod5_4",
    "tof_4",
    "barenco_tof_4",
    "tof_5",
    "mod_mult_55",
    "vbe_adder_3",
    "barenco_tof_5",
    "csla_mux_3",
    "rc_adder_6",
    "gf2^4_mult",
    "tof_10",
    "mod_red_21",
    "hwb6",
    "gf2^5_mult",
    "csum_mux_9",
    "barenco_tof_10",
    "qcla_com_7",
    "ham15-low",
    "gf2^6_mult",
    "qcla_adder_10",
    "gf2^7_mult",
    "gf2^8_mult",
    "qcla_mod_7",
    "adder_8",
]

path = "../circuits/nam/"


fns = [fn for fn in os.listdir(path) if isfile(join(path, fn)) and fn[-4:] == "qasm"]

output_dir = "./output/tket_nam/"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

result_dict = {}

for i, fn in enumerate(fns):
    full_path = path + fn
    gate_cnt, cnot_cnt, depth = tket_optimize_nam(full_path, output_dir)
    result_dict[fn] = [gate_cnt, cnot_cnt, depth]

csv_file = "./metricks/tket_nam.csv"

with open(csv_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "total_gates", "cx_gates", "depth"])
    for fn in circuit_order:
        if fn + ".qasm" in result_dict:
            writer.writerow([fn + ".qasm"] + result_dict[fn + ".qasm"])